In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import scipy
import pandas as pd
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from matplotlib.figure import Figure
import package_outlier as po
import math
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from scipy.stats import pearsonr
import itertools
import regex as re
from sklearn.metrics import roc_auc_score, r2_score


In [3]:
cwd_path = os.getcwd()
cwd_path

'/Users/saurabh/Desktop/HeartProjects/Exhalted/Params/Code_EXH/unexplained_dyspnea'

In [5]:
# reading file with feature names
feat_names_file = '../texture_3d_featnames.csv'
feat_names = pd.read_csv(feat_names_file, header=None)

In [5]:
feat_names.head()

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,median-Raw intensity,median-Gray mean ws=3,median-Gray median ws=3,median-Gray std_dev ws=3,median-Gray range ws=3,median-Gradient sobelx,median-Gradient sobely,median-Gradient sobelz,median-Gradient sobelxy,median-Gradient sobelyx,...,median-CoLlAGe O=2 idm ws=3,median-CoLlAGe O=2 correlation ws=3,median-CoLlAGe O=2 info1 ws=3,median-CoLlAGe O=2 info2 ws=3,median-CoLlAGe O=2 sum-av ws=3,median-CoLlAGe O=2 sum-var ws=3,median-CoLlAGe O=2 sum-ent ws=3,median-CoLlAGe O=2 diff-av ws=3,median-CoLlAGe O=2 diff-var ws=3,median-CoLlAGe O=2 diff-ent ws=3
1,var-Raw intensity,var-Gray mean ws=3,var-Gray median ws=3,var-Gray std_dev ws=3,var-Gray range ws=3,var-Gradient sobelx,var-Gradient sobely,var-Gradient sobelz,var-Gradient sobelxy,var-Gradient sobelyx,...,var-CoLlAGe O=2 idm ws=3,var-CoLlAGe O=2 correlation ws=3,var-CoLlAGe O=2 info1 ws=3,var-CoLlAGe O=2 info2 ws=3,var-CoLlAGe O=2 sum-av ws=3,var-CoLlAGe O=2 sum-var ws=3,var-CoLlAGe O=2 sum-ent ws=3,var-CoLlAGe O=2 diff-av ws=3,var-CoLlAGe O=2 diff-var ws=3,var-CoLlAGe O=2 diff-ent ws=3
2,kurtosis-Raw intensity,kurtosis-Gray mean ws=3,kurtosis-Gray median ws=3,kurtosis-Gray std_dev ws=3,kurtosis-Gray range ws=3,kurtosis-Gradient sobelx,kurtosis-Gradient sobely,kurtosis-Gradient sobelz,kurtosis-Gradient sobelxy,kurtosis-Gradient sobelyx,...,kurtosis-CoLlAGe O=2 idm ws=3,kurtosis-CoLlAGe O=2 correlation ws=3,kurtosis-CoLlAGe O=2 info1 ws=3,kurtosis-CoLlAGe O=2 info2 ws=3,kurtosis-CoLlAGe O=2 sum-av ws=3,kurtosis-CoLlAGe O=2 sum-var ws=3,kurtosis-CoLlAGe O=2 sum-ent ws=3,kurtosis-CoLlAGe O=2 diff-av ws=3,kurtosis-CoLlAGe O=2 diff-var ws=3,kurtosis-CoLlAGe O=2 diff-ent ws=3
3,skewness-Raw intensity,skewness-Gray mean ws=3,skewness-Gray median ws=3,skewness-Gray std_dev ws=3,skewness-Gray range ws=3,skewness-Gradient sobelx,skewness-Gradient sobely,skewness-Gradient sobelz,skewness-Gradient sobelxy,skewness-Gradient sobelyx,...,skewness-CoLlAGe O=2 idm ws=3,skewness-CoLlAGe O=2 correlation ws=3,skewness-CoLlAGe O=2 info1 ws=3,skewness-CoLlAGe O=2 info2 ws=3,skewness-CoLlAGe O=2 sum-av ws=3,skewness-CoLlAGe O=2 sum-var ws=3,skewness-CoLlAGe O=2 sum-ent ws=3,skewness-CoLlAGe O=2 diff-av ws=3,skewness-CoLlAGe O=2 diff-var ws=3,skewness-CoLlAGe O=2 diff-ent ws=3


In [6]:
# making list of patient ids
feat_vals_dir = '/Users/saurabh/texture_feature_stats/'
patient_dirs = glob.glob(feat_vals_dir+'*')
patient_ids = sorted([re.search('(EXH)(.+)',dir_name).group(2) for dir_name in patient_dirs])
print('total no. of patients =', len(patient_ids))

total no. of patients = 57


In [7]:
def populate_dfs(df, csv_file, stat):
    '''
    returns dataframe containing one statistical feature from either LV or myocardium.
    '''
    for ind, patient_id in enumerate(patient_ids):
        patient_dir = feat_vals_dir + 'EXH' + patient_id
        stat_data = pd.read_csv(patient_dir + f'/{csv_file}',header=None)
        df.loc[ind] = stat_data.iloc[stat].values
    df.dropna(axis='columns', inplace=True)    
    return df

In [8]:
def append_pkvo2(df):
    '''
    appends column containing pkvo2 values
    '''
    pids = df.patient_id.values
    pkvo2_vals = []
    for pid in pids:
        num_pid = int(pid[0:3])
        cpet_record = cpet_data[cpet_data['Record ID'].values == num_pid]
        pkvo2_vals.append(cpet_record['VO2 (MAX):'].values[0])

    df['pkvo2'] = pkvo2_vals
    
    return df, pkvo2_vals

In [9]:
def imp_gas_features_func(df, response, threshold):
    '''
    returns features that are most correlated with pkvo2
    '''
    texture_columns = df.columns
    predictors = texture_columns.drop(['patient_id', 'pkvo2']).values
    r2_vals = []
    for predictor in predictors:
        if predictor in df.columns.values:
            #print(predictor)
            if not df[predictor].isnull().sum(): 
                pearson_val = pearsonr(df[predictor],response)
                r2_vals.append((predictor, round(pearson_val[0]**2,3), "{:.2e}".format(pearson_val[1])))
    imp_gas_features = [ent for ent in sorted(r2_vals, key=lambda x: x[1], reverse=True) if ent[1] > threshold]
    #print('significant features: \n(feature name, R^2, p-value)\n', imp_gas_features)
    return imp_gas_features

In [10]:
# reading clinical data file containing pkvo2 values
cpet_file = '../ExerciseMRIEvaluatio_DATA_LABELS_2021-03-16_1304.xlsx'
cpet_data = pd.read_excel(cpet_file)
cpet_data.dropna(subset=['VO2 (MAX):'],inplace=True)
cpet_data.dropna(subset=['VO2 %Predicted '],inplace=True)
cpet_data.shape

(67, 662)

In [11]:
# making a list of important features
important_features = []
low_lv_ef = ['100B', '111', '114', '119', '123', '188']
for stat in range(4):
    col_names = feat_names.iloc[stat].values
    all_patients_lv_ed_stats_df = pd.DataFrame(columns = col_names)
    all_patients_lvmc_ed_stats_df = pd.DataFrame(columns = col_names)
    all_patients_lv_es_stats_df = pd.DataFrame(columns = col_names)
    all_patients_lvmc_es_stats_df = pd.DataFrame(columns = col_names)

    populate_dfs(all_patients_lv_ed_stats_df, 'feat_lv_ed_stats_3d.csv', stat)
    populate_dfs(all_patients_lvmc_ed_stats_df, 'feat_lvmc_ed_stats_3d.csv', stat)    
    populate_dfs(all_patients_lv_es_stats_df, 'feat_lv_es_stats_3d.csv', stat)    
    populate_dfs(all_patients_lvmc_es_stats_df, 'feat_lvmc_es_stats_3d.csv', stat)
    
    all_patients_lv_ed_stats_df['patient_id'] = patient_ids
    all_patients_lvmc_ed_stats_df['patient_id'] = patient_ids
    all_patients_lv_es_stats_df['patient_id'] = patient_ids
    all_patients_lvmc_es_stats_df['patient_id'] = patient_ids
    
    all_patients_lv_ed_stats_df = all_patients_lv_ed_stats_df[~all_patients_lv_ed_stats_df['patient_id'].isin(low_lv_ef)]
    all_patients_lvmc_ed_stats_df = all_patients_lvmc_ed_stats_df[~all_patients_lvmc_ed_stats_df['patient_id'].isin(low_lv_ef)]
    all_patients_lv_es_stats_df = all_patients_lv_es_stats_df[~all_patients_lv_es_stats_df['patient_id'].isin(low_lv_ef)]
    all_patients_lvmc_es_stats_df = all_patients_lvmc_es_stats_df[~all_patients_lvmc_es_stats_df['patient_id'].isin(low_lv_ef)]
    
    for ind, df in enumerate([all_patients_lv_ed_stats_df.copy(), all_patients_lvmc_ed_stats_df.copy(), \
                           all_patients_lv_es_stats_df.copy(), all_patients_lvmc_es_stats_df.copy()]):
        df_cols = df.columns
        numerical_columns = df_cols.drop(['patient_id']).values    
        df, pkvo2_vals = append_pkvo2(df)      
        
        df_scaled = df.copy()
        df_scaled[numerical_columns] = scaler.fit_transform(df[numerical_columns])
        
        imp_pkvo2_features = imp_gas_features_func(df_scaled, pkvo2_vals, 0.1)
        if not imp_pkvo2_features == []:
            important_features.append(((stat, ind), [ent[0] for ent in imp_pkvo2_features]))

In [12]:
def read_csvs(df, csv_file, feats, stat):
    '''
    returns selected features from a dataframe
    '''
    row = stat
    for i, feat in enumerate(feats):
        column = list(feat_names.iloc[stat].values).index(feat)
        data = []
        for j, patient_id in enumerate(patient_ids):
            patient_dir = feat_vals_dir + 'EXH' + patient_id
            stat_data = pd.read_csv(patient_dir + f'/{csv_file}',header=None)
            data.append(stat_data.iloc[row][column])
        df[feat] = data
    return df

In [13]:
# dictionary with key to each LV/myocardium, ED/ES file
csv_dict = {
           0: 'feat_lv_ed_stats_3d.csv', \
           1: 'feat_lvmc_ed_stats_3d.csv', \
           2: 'feat_lv_es_stats_3d.csv', \
           3: 'feat_lvmc_es_stats_3d.csv'
            }
important_texture_features_df = pd.DataFrame()

In [14]:
# reading the important features from texture files of each patient
for file in important_features:
    features = file[1]
    csv_file = csv_dict[file[0][1]]
    #print(features, csv_file, file[0][0])
    read_csvs(important_texture_features_df, csv_file, features, file[0][0])

In [25]:
important_texture_features_df.head()

,median-Laws E3E3S3,median-Laws S3S3L3,median-Gray std_dev ws=3,median-Laws L3S3L3,median-Gradient sobelyz,median-Gradient sobelzx,median-Gradient magnitude,median-Gradient sobelxz,median-Laws L3S3S3,median-Gradient sobelzy,...,"skewness-Gabor XY-θ=1.571, XZ-θ=1.571, λ=0.880, BW=1","skewness-Gabor XY-θ=1.963, XZ-θ=1.571, λ=0.880, BW=1","skewness-Gabor XY-θ=2.356, XZ-θ=1.571, λ=0.880, BW=1","skewness-Gabor XY-θ=2.749, XZ-θ=1.571, λ=0.880, BW=1","skewness-Gabor XY-θ=1.571, XZ-θ=0.785, λ=0.880, BW=1","skewness-Gabor XY-θ=0.000, XZ-θ=2.356, λ=0.880, BW=1","skewness-Gabor XY-θ=0.000, XZ-θ=0.785, λ=0.880, BW=1",skewness-Laws S3S3S3,patient_id,pkvo2
1,0.0,3.0,32.966292,1018.0,-75.0,24.0,31.721444,143.0,6.0,-59.0,...,0.132500,0.132500,0.132500,0.132500,0.173203,0.184406,0.191166,0.351876,101B,1.52
2,-1.0,10.0,25.971547,388.0,-271.0,351.0,24.929902,221.0,10.0,-332.0,...,-0.231362,-0.231362,-0.231362,-0.231362,-0.060460,-0.087546,-0.052435,0.248318,103,2.36
3,0.0,6.0,17.474517,309.5,-9.0,59.5,16.120601,-7.5,3.0,-50.0,...,-0.762869,-0.762869,-0.762869,-0.762869,-0.648606,-0.652213,-0.653317,0.579568,105B,2.53
4,0.0,32.0,36.861138,790.0,-459.0,383.0,37.569935,410.0,15.0,-469.0,...,0.088654,0.088654,0.088654,0.088654,0.208908,0.209827,0.239826,0.097266,109,1.23
6,0.0,27.0,56.758966,1851.0,-757.0,822.0,37.218947,450.0,5.0,-618.0,...,0.275969,0.275969,0.275969,0.275969,0.441740,0.531490,0.360195,0.464840,113B,1.32


In [16]:
# extracting numerical columns
numerical_columns = important_texture_features_df.columns.values

In [19]:
# dropping low LVEF patients
important_texture_features_df['patient_id'] = patient_ids
important_texture_features_df,_, = append_pkvo2(important_texture_features_df)
important_texture_features_df = important_texture_features_df[~important_texture_features_df['patient_id'].isin(low_lv_ef)]


In [22]:
# looking at texture features with most correlation with PkVO2
important_texture_features_df_scaled = important_texture_features_df.copy()
important_texture_features_df_scaled[numerical_columns] = scaler.fit_transform(important_texture_features_df[numerical_columns])        
imp_pkvo2_features = imp_gas_features_func(important_texture_features_df_scaled, pkvo2_vals, 0.12)
imp_pkvo2_features

[('median-Laws  S3S3L3', 0.182, '1.81e-03'),
 ('skewness-Laws  S3S3L3', 0.162, '3.36e-03'),
 ('median-Gray std_dev ws=3', 0.157, '3.98e-03'),
 ('skewness-Laws  S3L3L3', 0.155, '4.20e-03'),
 ('median-Laws  L3S3L3', 0.152, '4.63e-03'),
 ('var-Gradient z', 0.148, '5.28e-03'),
 ('skewness-Laws  L3L3S3', 0.141, '6.59e-03'),
 ('median-Gradient sobelyz', 0.138, '7.37e-03'),
 ('median-Gradient sobelzx', 0.136, '7.81e-03'),
 ('skewness-Gradient sobelxy', 0.135, '8.08e-03'),
 ('var-Laws  E3E3L3', 0.131, '8.92e-03'),
 ('var-Laws  E3S3L3', 0.129, '9.56e-03'),
 ('median-Gradient magnitude', 0.127, '1.04e-02'),
 ('var-Gabor XY-θ=1.178, XZ-θ=0.000, λ=0.880, BW=1', 0.127, '1.02e-02'),
 ('var-Gabor XY-θ=1.963, XZ-θ=0.000, λ=0.880, BW=1', 0.127, '1.04e-02'),
 ('median-Gradient sobelxz', 0.126, '1.08e-02'),
 ('median-Laws  E3E3S3', 0.125, '1.09e-02'),
 ('kurtosis-Gabor XY-θ=2.356, XZ-θ=0.785, λ=0.880, BW=1', 0.124, '1.14e-02'),
 ('kurtosis-CoLlAGe O=2 sum-var ws=3', 0.123, '1.15e-02'),
 ('median-Laws  L3

In [23]:
# saving dataframe to csv
important_texture_features_df.to_csv('important_texture_3d_features.csv')